In [43]:
# Import Dependencies
import pandas as pd
import numpy as np
import re
import spacy
import sqlite3
from spacy.attrs import ORTH, LEMMA, NORM, TAG
from datetime import datetime, timedelta
from spacy.attrs import ORTH, LEMMA, NORM, TAG
from clean import replace_emoticons, clean_text, clean_tweet
from joblib import load

In [100]:
# Load machine learning prediction model
model = load("twitter_model.joblib")

In [112]:
username = "@JoeBiden"

In [ ]:
# Load tweets and apply model

In [ ]:
# Import candidate's tweets into a dataframe
twitter_df = pd.read_csv(f"data/{username}_clean.csv", error_bad_lines=False)

In [ ]:
twitter_df.head()

In [ ]:
# Predict sentiment based on model and put into new sentiment column
twitter_df["sentiment"] = twitter_df["tweet"].map(lambda x: model.predict(x))

In [ ]:
# Create username column with candidate's username
twitter_df["username"] = username

In [ ]:
# Set index to be id column
twitter_df = twitter_df.set_index('id')

In [ ]:
twitter_df.head()

In [ ]:
# Connect to database and create twitter table
conn = sqlite3.connect("politics_db.db")
conn.execute("""CREATE TABLE "twitter" (
            "id" INTEGER UNIQUE,
            "tweet" TEXT, 
            "tweet_date" TEXT,
            "retweet" INTEGER,
            "favorite" INTEGER,
            "username" TEXT,
            "sentiment" INTEGER     
            )""")

In [ ]:
# Add candidate's tweets to twitter table
twitter_df.to_sql("twitter", conn, if_exists="append")